In [1]:
!pip install influxdb_client influxdb

In [2]:
#First add kafka consumer to consume dataset

In [127]:
import numpy as np
import random
import pymongo
from pyspark.sql import SparkSession
import pandas as pd
import influxdb_client
from influxdb_client.client.write_api import SYNCHRONOUS

In [3]:
spark = SparkSession.builder.appName("Test").getOrCreate()
df = pd.read_excel('SWaT_dataset_Jul 19 v2.xlsx', parse_dates = ['GMT +0']) #remove later
sdf = spark.createDataFrame(df)
sdf.show()

/usr/local/spark/python/pyspark/sql/pandas/conversion.py:474: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():
/usr/local/spark/python/pyspark/sql/pandas/conversion.py:486: FutureWarning: iteritems is deprecated and will be removed in a future version. Use .items instead.
  for column, series in pdf.iteritems():


+--------------------+-----------+----------+------+--------+-----------+-----------+----------+--------+----------+----------+--------+--------+--------+--------+-----+--------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+-----------+--------+----------+----------+----------+----------+----------+------+------+------+------+--------+-----------+-----------+-------+----------+-----------+----------+--------+--------+-----------+-----------+-----------+-----------+-----+----------+----------+----------+----------+-----------+-----------+-----------+----------+------+------+------+------+--------+-----------+-----------+----------+----------+----------+----------+-------+-------+--------+--------+--------+-------+--------+-----------+-----------+-----------+
|              GMT +0|    FIT 101|   LIT 101|MV 101|P1_STATE|P101 Status|P102 Status|   AIT 201| AIT 202|   AIT 203|   FIT 201|  LS 201|  LS 202| LSL 203|LSLL 203|MV201|P2_STATE|P201 Status|P202

# WavingSketch
Finding Frequent Items, Finding Heavy Changes, Finding Persistent Items, Finding Super-Spreaders

In [ ]:
"""
wavingCounter unbiased estimation of frequencies
wavingList store k' (k'>k) frequent items
hashFunc to hash element to 1 or -1 increase or decrease wavingCounter
element frequency
If its frequency is larger than the smallest frequency in the list,we exchange them. 

Based on this idea, we use a group of WavingCounters and lists, and add additional fields in the list to achieve
higher accuracy
"""

In [61]:
class WavingSketch:
    def __init__(self, size):
        self.arr = np.zeros((size,),dtype='f,f,f,f')
        self.count = 0
    def update(self,elem):
        round(elem, 7)
        # bucket[0]-->count bucket[1]-->elem bucket[2]-->frequency bucket[3]-->flag
        bucket = self.arr[self.hashnum(elem)]
        # estimated frequency calc
        efreq = bucket[0]*self.hash1(elem)
        if elem == bucket[1]:
            bucket[2]+=1
            if bucket[3]==False:
                bucket[0]=bucket[0]+self.hash1(elem)
        elif bucket[1] == 0:
            bucket[1]=elem
            bucket[2]=1
            bucket[3]=True
        else:
            bucket[0]=bucket[0]+self.hash1(elem)
            if efreq >= bucket[0]:
                if bucket[3]==True:
                    bucket[0]=bucket[0]+efreq*self.hash1(elem)
                bucket[1]=elem
                bucket[2]=efreq+1
                bucket[3]=False
            
    def hash1(self,elem):
        val = -1
        if self.hashnum(elem)%2==0:
            val = 1
        return val
    def hashnum(self,elem):
        return hash(elem.__str__()) % len(self.arr)
    
    def ret(self):
        return self.arr

In [111]:
def findkfreq(arr,k):
    if k > len(arr):
        return -1
    freq=[]
    for i in arr:
        freq.append((i[0],i[1]))
    freq.sort(reverse = True)
    freq = freq[0:k]
    return freq
    #return [i[1] for i in freq]

def findChange(arr1,arr2,k):
    freq1=[]
    freq2=[]
    for i in arr1:
        freq1.append((i[0],i[1]))
    for j in arr2:
        freq2.append((j[0],j[1]))
    freqres=[]
    for i in range(len(freq1)):
        freqres.append(((freq2[i][0]-freq1[i][0],freq1[i][1])))
    freqres.sort(reverse = True)
    freqres = freqres[0:k]
    return freqres
    #return [i[1] for i in freqres]
    

In [115]:
sketch= WavingSketch(30)
attarr = np.array(sdf.select("FIT 401").collect())
for elem in attarr:
    sketch.update(elem[0])
samples=sketch.ret()

freq5 = findkfreq(samples,5)

In [116]:
sketch1 = WavingSketch(30)
sketch2 = WavingSketch(30)
attarr1 = attarr[0:int(len(attarr)/2)]
attarr2 = attarr[int(len(attarr)/2):len(attarr)]

for elem in attarr1:
    sketch1.update(elem[0])
for elem in attarr2:
    sketch2.update(elem[0])
samples1=sketch1.ret()
samples2=sketch2.ret()

change5 = findChange(samples1,samples2,5)

Store in Mongo

In [122]:
#myclient = pymongo.MongoClient("mongodb://mongo:27017/")
conn = pymongo.MongoClient('mongodb://mongo:27017')

mydb = conn["mydatabase"]

In [125]:
mycol = mydb["FIT 401 frequency"]
my_list=[]
cont=1
for i in freq5:
    my_list.append({ "_id": cont, "value": i[1], "frequency": i[0]})
    cont+=1

print(my_list)
x = mycol.insert_many(mylist)

[{'_id': 1, 'value': 0.80799025, 'frequency': 876.0}, {'_id': 2, 'value': 0.80927074, 'frequency': 802.0}, {'_id': 3, 'value': 0.8056854, 'frequency': 794.0}, {'_id': 4, 'value': 0.8031244, 'frequency': 783.0}, {'_id': 5, 'value': 0.8028683, 'frequency': 669.0}]


InfluxDB

In [128]:
token = "i4mHlTHGTjH0nK974IYaZJztxhOdhwPhmcdYvZTVom05ZyrsJqwNLT3_6EUWPwi97nCG0oNx2QjMTlRapn5Q1g=="
org = "changeme"
bucket = "frequency"


with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    for freqs in freq5:
        
        p = influxdb_client.Point("WavingFreq FIT 401").field(str(freqs[1]), freqs[0])
        write_api.write(bucket, org, p)    
            

In [129]:
token = "i4mHlTHGTjH0nK974IYaZJztxhOdhwPhmcdYvZTVom05ZyrsJqwNLT3_6EUWPwi97nCG0oNx2QjMTlRapn5Q1g=="
org = "changeme"
bucket = "frequency"


with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    for freqs in change5:
        
        p = influxdb_client.Point("WavingChange FIT 401").field(str(freqs[1]), freqs[0])
        write_api.write(bucket, org, p)    
            

## Reservoir Sampling

In [42]:
class ReservoirSampling:
    def __init__(self, size):
        self.arr = np.zeros(size)
        self.count = 0
    def update(self,elem):
        # if counter less than the pool size we add this element in the first free position 
        if self.count < len(self.arr):
            for i in range(len(self.arr)):
                if self.arr[i]==0:
                    self.arr[i] = elem
                    self.count += 1
        # it counter >= pool size then we have to apply update algorithm
        r = random.randint(0, self.count-1)
        if r < len(self.arr):
            self.arr[r]=elem
        self.count += 1
    
    def ret(self):
        return self.arr

In [44]:
token = "i4mHlTHGTjH0nK974IYaZJztxhOdhwPhmcdYvZTVom05ZyrsJqwNLT3_6EUWPwi97nCG0oNx2QjMTlRapn5Q1g=="
org = "changeme"
bucket = "freq"

attarr = np.array(sdf.select("FIT 401").collect())
sampling1 = ReservoirSampling(20)
for elem in attarr:
    sampling1.update(elem)
samples=sampling1.ret()

cont=0


with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    for attack in attacks:
        for ob in samples:
            p = influxdb_client.Point("sampling FIT 401").field(str(ob), ob)
            write_api.write(bucket, org, p)    
            
            


## Misra Gries Algorithm
Finding frequent items determinastically. Given a bag b of n elements and an integer k ≥ 2. Find the values that occur more than n ÷ k times in b

In [38]:
def misra(arr, k):
    dic = {}
    for elem in arr:
        ## case 1: item already has counter or there are empty counters
        if elem in dic:
            dic[round(elem, 7)] +=1
        elif len(dic)<k:
            dic[round(elem, 7) ] = 1
        ## case 2: item doesn't have counter and there are no empty counters
        else:
            for key in list(dic.keys()):
                dic[round(key, 7)] -= 1
                if dic[round(key, 7)] == 0:
                    del dic[round(key, 7)]
    return dic



In [39]:
attack1 = np.array(sdf.select("FIT 401").collect())
flat_list = [item for sublist in attack1 for item in sublist]

In [40]:
data = misra(flat_list,15)

In [41]:
token = "i4mHlTHGTjH0nK974IYaZJztxhOdhwPhmcdYvZTVom05ZyrsJqwNLT3_6EUWPwi97nCG0oNx2QjMTlRapn5Q1g=="
org = "changeme"
bucket = "freq"

with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    dataCollect = sdf.collect()
    for row in dataCollect:
        for key in data:
            p = influxdb_client.Point("freqFIT401").field(key, data[key])
            write_api.write(bucket, org, p)    
            
            


KeyboardInterrupt: 

# BJKST Algorithm
Count distinct elements

In [36]:
import math
class BJKST:
    def __init__(self, n, eps, c):
        self.h = UniversalHash(n)
        self.g = UniversalHash(int(math.log(n, 2)**2 * eps**(-4)))
        self.z = 0
        self.b = []
        self.eps = eps 
        self.c = c
    def update(self,elem):
        # if counter less than the pool size we add this element in the first free position 
        nzeros = np.count_nonzero(self.h.hash(elem)==0)
        if nzeros >= self.z:
            self.b.append([self.g.hash(elem) , nzeros])
        while len(self.b) >= self.c/self.eps**2:
            self.z += 1
            for i in self.b:
                if i[1] < self.z:
                    self.b.remove(i)
                    
    def ret(self):
        return len(self.b)*2**self.z

class UniversalHash:

    def __init__(self, space):
        self.random = random.Random()
        self.random.seed()
        self.space = space
    
    def hash(self,key):
        key = -(key + 1) if key < 0 else key
        return hash(key.__str__()) % self.space
    

In [37]:
token = "i4mHlTHGTjH0nK974IYaZJztxhOdhwPhmcdYvZTVom05ZyrsJqwNLT3_6EUWPwi97nCG0oNx2QjMTlRapn5Q1g=="
org = "changeme"
bucket = "freq"

bjkst1 = BJKST(20,1,3)
attack1 = np.array(sdf.select("LIT 401").collect())
flat_list = [item for sublist in attack1 for item in sublist]

with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    dataCollect = sdf.collect()
    for i in flat_list:
        bjkst1.update(i)
        val = bjkst1.ret()
        p = influxdb_client.Point("difcountlIT401").field(str(val), val)
        write_api.write(bucket, org, p)    
        #time.sleep(5)

# Morris counter

In [33]:
def morris(arr):
    cont = 0
    c = 10
    for elem in arr:
        cont += 2**(-c)
    return (2**cont)-1

In [35]:
attacks = ["FIT 401","LIT 401"]

token = "i4mHlTHGTjH0nK974IYaZJztxhOdhwPhmcdYvZTVom05ZyrsJqwNLT3_6EUWPwi97nCG0oNx2QjMTlRapn5Q1g=="
org = "changeme"
bucket = "freq"

with influxdb_client.InfluxDBClient(url="http://influxdb:8086", token=token, org=org) as client:
    write_api = client.write_api(write_options=SYNCHRONOUS)
    for attack in attacks:
        attarr = np.array(sdf.select(attack).collect())
        p = influxdb_client.Point("counter"+attack).field("counter"+attack, morris(attarr))
        write_api.write(bucket, org, p)    
            
            

25610.981454890723
25610.981454890723


In [ ]:
"""
def wingraphminer(stream, batch, windowsize, support):
    g = []
    q = []
    window = []
    R= []
    for i in range(0,len(stream),batch):
        data = stream[i:batch]
        c = coreset(data,support)
        window.append(c)
        if len(window) >= windowsize:
            R = window[0]
        else:
            R = []

def coreset(batch,support):
    ADS
"""             